# Task 1

In [ ]:
library(dplyr)
library(tidyr)
library(data.table)

In [2]:
# path to the gene_info file
gene_info_file_path <- "/content/Homo_sapiens.gene_info.gz"

# Read the tab-delimited file with gzip compression and get required columns
gene_info_df <- fread(gene_info_file_path, sep='\t', header=TRUE)[, .(GeneID, Symbol, Synonyms)]

In [3]:
head(gene_info_df)

GeneID,Symbol,Synonyms
<int>,<chr>,<chr>
1,A1BG,A1B|ABG|GAB|HYST2477
2,A2M,A2MD|CPAMD5|FWP007|S863-7
3,A2MP1,A2MP
9,NAT1,AAC1|MNAT|NAT-1|NATI
10,NAT2,AAC2|NAT-2|PNAT
11,NATP,AACP|NATP1


In [4]:
print(head(gene_info_df))

   GeneID Symbol                  Synonyms
1:      1   A1BG      A1B|ABG|GAB|HYST2477
2:      2    A2M A2MD|CPAMD5|FWP007|S863-7
3:      3  A2MP1                      A2MP
4:      9   NAT1      AAC1|MNAT|NAT-1|NATI
5:     10   NAT2           AAC2|NAT-2|PNAT
6:     11   NATP                AACP|NATP1


In [5]:
# Separate items in the Synonyms column into new rows

new_data1<-gene_info_df %>%
  separate_rows(Synonyms, sep="\\|")
head(new_data1)

GeneID,Symbol,Synonyms
<int>,<chr>,<chr>
1,A1BG,A1B
1,A1BG,ABG
1,A1BG,GAB
1,A1BG,HYST2477
2,A2M,A2MD
2,A2M,CPAMD5


In [6]:
# Mapping the Symbol to their Gene ID
map_Symbol <- with(gene_info_df, setNames(GeneID, Symbol))

# Mapping the Synonyms to their Gene ID
map_Synonyms <- with(new_data1, setNames(GeneID, Synonyms))

# combined two vectors with key as the gene name and value is the Gene ID
map_Symbol_Synonyms_geneid <- c(map_Symbol, map_Synonyms)

In [7]:
map_Symbol_Synonyms_geneid["GAB"]

GAB 
  1

# Task 2

In [8]:
# Path to the gmt file
input_gmt_path <- "/content/h.all.v2023.1.Hs.symbols.gmt"

out_gmt_file <- "output_file.gmt"

out_gmt_file <- file(out_gmt_file, "w")

# Read the input gmt file line by line

inp_file <- file(input_gmt_path, "r")

while (length(line <- readLines(inp_file, n = 1, warn = FALSE)) > 0) {
  pathway_info <- unlist(strsplit(line, "\t"))

  pathway_name <- pathway_info[1]
  pathway_desc <- pathway_info[2]
  gene_names <- pathway_info[-(1:2)]  # All subsequent values are gene names


  entrez_ids <- sapply(gene_names, function(gene) {
    if (gene %in% names(map_Symbol_Synonyms_geneid)) {
      return(map_Symbol_Synonyms_geneid[[gene]])
    } else {
      return(gene)  # If Entrez ID not found, keep the original gene name
    }
  })

  # Concatenate pathway information with entrez_ids
  pathway_line <- c(pathway_name, pathway_desc, entrez_ids)

  # Write updated pathway information to the output file
  cat(paste(pathway_line, collapse = "\t"), file = out_gmt_file, sep = "\n")
}

close(inp_file)
close(out_gmt_file)